- calcular a distancia de todos os consultores a todas as escolas
- colocar um valor aleatorio de (b) para cada escola (rand between 0.6 e 0.9)

In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import plotly.express as px


import warnings
warnings.filterwarnings('ignore')


In [2]:
df_escolas = pd.read_csv("../dados/temporarios/10-md_edb_cols_finais.csv",
                 usecols=["CO_ENTIDADE", "valor_venda", "lat", "lng"])

df_escolas.head()

,CO_ENTIDADE,valor_venda,lat,lng
0,11006765,567134.87,-9.91131,-63.04099
1,11007028,173815.59,-9.89796,-63.04024
2,11007656,888919.23,-9.90869,-63.03561
3,11044454,173253.08,-9.90840,-63.03315
4,11044462,953800.11,-9.89198,-63.03205


## Obtendo distancias

In [3]:
df_consultores = pd.read_csv("../dados/temporarios/consultor_coords.csv")

df_consultores.head()

,Consultor,lat,lng
0,Arthur Lincoln,-22.96851,-43.18552
1,Ana Claudia,-23.21074,-45.85701
2,Elina Maciel,-22.87912,-47.03464
3,Galdino Espindola,-22.91447,-43.05093
4,Niedna Leite,-23.59439,-46.61307


In [4]:
def calcula_distancias():
    pre_df = {"CO_ENTIDADE":df_escolas["CO_ENTIDADE"]}
    for _, consultor in df_consultores.iterrows():
        nome_co = consultor["Consultor"]
        lat_co = consultor["lat"]
        lon_co = consultor["lng"]

        dists = []
        for _, escola in df_escolas.iterrows():
            lat_es = escola["lat"]
            lon_es = escola["lng"]
            dists.append(round(geodesic((lat_es, lon_es), (lat_co, lon_co)).km))
        
        pre_df.update({nome_co: dists})

    df_dists = pd.DataFrame(pre_df)
    df_dists.to_csv("../dados/temporarios/30-dist_cons_esco.csv", index=False)

    return df_dists

# b = calcula_distancias()

## Obtendo valor de (a)
por enquanto usando random

In [5]:
df_afinidade = df_escolas[["CO_ENTIDADE", "valor_venda"]]

In [6]:
np.random.seed(2025)

df_afinidade["afinidade"] = np.random.normal(0.9, 0.01, size=len(df_afinidade))


df_afinidade["afinidade"]
print(min(df_afinidade["afinidade"]), max(df_afinidade["afinidade"]))

# sum(df_afinidade["afinidade"] > 1)

0.8603226626011911 0.9376087003196701


In [18]:

fig = px.histogram(df_afinidade["afinidade"])

fig.show()

## Combinando (a.b)

In [20]:
df_final = df_afinidade.copy()


df_final["valor_ponderado"] = ((df_afinidade["afinidade"] * df_afinidade["valor_venda"])/100).astype(int)



df_final.head()

,CO_ENTIDADE,valor_venda,afinidade,valor_ponderado
0,11006765,567134.87,0.899076,5098
1,11007028,173815.59,0.907343,1577
2,11007656,888919.23,0.885611,7872
3,11044454,173253.08,0.893366,1547
4,11044462,953800.11,0.898993,8574


In [21]:

fig = px.histogram(df_final["valor_ponderado"])

fig.show()

In [22]:
df_final.to_csv("../dados/temporarios/30-pronto_PO.csv", index=False)